### Piano

Run this notebook and see what happens. 

Click the game canvas once it shows up to bring it into focus.

* Keyboard image was adapted from [jack-keyboard](http://jack-keyboard.sourceforge.net/).

In [1]:
# Creative Commons sound samples:
# http://wiki.laptop.org/go/Sound_samples
# https://freepats.zenvoid.org/

In [2]:
import logging
import rtmidi
import mido
import sys
import os

import numpy as np

In [3]:
sys.path.insert(0, os.path.abspath('./..'))

In [4]:
import jupylet.color

from jupylet.app import App
from jupylet.sound import Sample, Synth, get_oscilloscope_as_image
from jupylet.sound import use, play, sleep
from jupylet.state import State
from jupylet.label import Label
from jupylet.sprite import Sprite

In [5]:
logger = logging.getLogger()

In [6]:
app = App(width=512, height=420)#, log_level=logging.INFO)

In [7]:
a0 = np.zeros((256, 512, 4), 'uint8')

In [8]:
oscilloscope = Sprite(a0, x=256, y=292)

In [9]:
layout = Sprite('images/keyboard.png', x=256, y=82, scale=0.5)

In [10]:
synth = Synth()

In [11]:
#synth.set_envelope(min_duration=0.05, attack=0.0, decay=0.1, sustain=0.7, release=0.5)

In [12]:
keys = app.window.keys

keyboard = {

    keys.Z: 'C',
    keys.S: 'Cs',
    keys.X: 'D',
    keys.D: 'Ds',
    keys.C: 'E',
    keys.V: 'F',
    keys.G: 'Fs',
    keys.B: 'G',
    keys.H: 'Gs',
    keys.N: 'A',
    keys.J: 'As',
    keys.M: 'B',

    keys.Q: 'C5',
    50: 'Cs5',
    keys.W: 'D5',
    51: 'Ds5',
    keys.E: 'E5',
    keys.R: 'F5',
    53: 'Fs5',
    keys.T: 'G5',
    54: 'Gs5',
    keys.Y: 'A5',
    55: 'As5',
    keys.U: 'B5',

    keys.I: 'C6',
    keys.NUMBER_9: 'Cs6',
    keys.O: 'D6',
    48: 'Ds6',
    keys.P: 'E6',
}

In [13]:
state = State(
    
    amp = 1.,
    ms = 50.,
    
    up = False,
    down = False,
    left = False,
    right = False,
)

In [14]:
label0 = Label('amp: %.1f' % state.amp, x=10, y=194)
label1 = Label('span: %.1f ms' % state.ms, x=10, y=174)
label2 = Label('use ← → ↑ ↓ to modify', anchor_x='right', x=app.width - 10, y=174)

In [15]:
pk = {}

In [16]:
@app.event
def key_event(key, action, modifiers):
            
    keys = app.window.keys
    value = action == keys.ACTION_PRESS

    if key == keys.UP:
        state.up = value

    if key == keys.DOWN:
        state.down = value

    if key == keys.RIGHT:
        state.right = value

    if key == keys.LEFT:
        state.left = value
        
    if action == keys.ACTION_PRESS and key in keyboard:
        assert key not in pk
        pk[key] = synth.play_new(note=keyboard[key])
        
    if action == keys.ACTION_RELEASE and key in keyboard:
        pk.pop(key).play_release()

In [17]:
@app.run_me_every(1/24)
def modify_oscilloscope(ct, dt):
    
    s = 2 ** dt
    
    if state.up:
        state.amp *= s
        label0.text = 'amp: %.1f' % state.amp

    if state.down:
        state.amp /= s
        label0.text = 'amp: %.1f' % state.amp

    if state.right:
        state.ms *= s
        state.ms = min(256, state.ms)
        label1.text = 'span: %.1f ms' % state.ms

    if state.left:
        state.ms /= s
        label1.text = 'span: %.1f ms' % state.ms

In [18]:
@app.event
def render(ct, dt):
    
    app.window.clear(color='#555')
    
    im, ts, te = get_oscilloscope_as_image(
        1/app.interval,
        ms=state.ms, 
        amp=state.amp, 
        color=255, 
        size=(512, 256)
    )
    
    oscilloscope.image = im    
    oscilloscope.draw()
    
    layout.draw()
    
    label0.draw()
    label1.draw()
    label2.draw()

In [19]:
xylo = Sample('sounds/VCSL/Xylophone/Xylophone - Medium Mallets.sfz')
xylo.amp = 8

In [20]:
_keyd = {}

In [30]:
@app.event
def midi_message(msg):
    
    if msg.type == 'note_on':
        
        if msg.velocity != 0:
            _keyd[msg.note] = xylo.play_new(key=msg.note, velocity=msg.velocity)
            
        elif msg.note in _keyd:
            _keyd[msg.note].play_release()

In [22]:
#app.get_logging_widget()

In [23]:
app.run()

Image(value=b'\xff\xd8\xff\xe0\x00\x10JFIF\x00\x01\x01\x00\x00\x01\x00\x01\x00\x00\xff\xdb\x00C\x00\x08\x06\x0…

In [24]:
#
# Uncomment and run to change sound volume:
#
#synth.amp = 0.2
#

In [22]:
from jupylet.sound import note

In [23]:
@app.sonic_live_loop(times=10)
async def loop0():
            
    use(synth, duration=1.7, shape='saw', amp=0.4)

    play('C2')
    await sleep(3.)

    play('E2')
    await sleep(3.)

    play('C2')
    await sleep(6.)


In [24]:
@app.sonic_live_loop
async def loop1():
    
    use(synth, amp=0.3)
        
    play(note.C5)
    await sleep(1.)

    play(note.E5)
    await sleep(1.)

    play(note.G5)
    await sleep(1.)

In [ ]:
#
# Uncomment and run to stop the live loops.
#
#app.stop(loop0)
#app.stop(loop1)
#